In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# import zipfile
# with zipfile.ZipFile("/content/drive/MyDrive/iseg2019/dataISEG_np/train_patch32_stride8.zip", 'r') as zip_ref:
#     zip_ref.extractall("data_train")
# with zipfile.ZipFile("/content/drive/MyDrive/iseg2019/dataISEG_np/test_patch32_stride8.zip", 'r') as zip_ref:
#     zip_ref.extractall("data_test")

Mounted at /content/drive


In [ ]:
from IPython.display import clear_output
import nibabel as nib
import glob 
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from fastprogress import master_bar, progress_bar
from torchvision import transforms
import torch.optim as optim
from itertools import product
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
clear_output()

def csv_log(filepath,**dict_save):
    header = list(dict_save.keys())
    isFile = os.path.isfile(filepath)
    with open(filepath, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=header)
        if not isFile:
            writer.writeheader()
        writer.writerow(dict_save) 
def extract_patches(arr, patch_shape=8, extraction_step=1):
    arr_ndim = arr.ndim
    patch_strides = arr.strides

    slices = [slice(None, None, st) for st in extraction_step]
    indexing_strides = arr[tuple(slices)].strides

    patch_indices_shape = ((np.array(arr.shape) - np.array(patch_shape)) //
                           np.array(extraction_step)) + 1
    
    shape = tuple(list(patch_indices_shape) + list(patch_shape))
    strides = tuple(list(indexing_strides) + list(patch_strides))
    patches = np.lib.stride_tricks.as_strided(arr, shape=shape, strides=strides)
    npatches = np.prod(patches.shape[:arr_ndim])
    return patches.reshape((npatches, ) + patch_shape)

def reconstruct_volume_avg(patches, expected_shape, extraction_step = (1,8,8,8), num_class = 4):
    v_x, v_y, v_z = expected_shape
    p_x, p_y, p_z = patches.shape[2:]
    s_x, s_y, s_z = extraction_step[1:]# compute the dimensions of the patches array
    n_x = (v_x - p_x) // s_x + 1
    n_y = (v_y - p_y) // s_y + 1
    n_z = (v_z - p_z) // s_z + 1

    vol = np.zeros((num_class, v_x, v_y, v_z))
    count = np.zeros((num_class, v_x, v_y, v_z))

    for p, (i, j, k) in zip(patches, product(range(n_x), range(n_y), range(n_z))):
        vol[:, i*s_x:i*s_x+ p_x, j*s_y:j*s_y + p_y, k*s_z:k*s_z + p_z]  +=p
        count[:, i*s_x:i*s_x+ p_x, j*s_y:j*s_y + p_y, k*s_z:k*s_z + p_z] +=1
    return vol/count

def min_max_preprocess(image, low_perc=1, high_perc=99):
    """Main pre-processing function used for the challenge (seems to work the best).
    Remove outliers voxels first, then min-max scale.
    """
    non_zeros = image > 0
    low, high = np.percentile(image[non_zeros], [low_perc, high_perc])
    image = np.clip(image, low, high)
    image = (image - low) / (high - low)
    return image
class AddNoise():
    def __init__(self, p):
        self.p = p
    def __call__(self, volume):
        if self.p > torch.rand(1):
            volume = volume * np.random.uniform(0.9, 1.1)
            std_per_channel = np.stack([np.std(volume[channel][volume[channel] > 0]) \
                                        for channel in range(volume.shape[0])])
            noise = np.concatenate([np.random.normal(0, std * 0.1+1e-5, size=volume[:1].shape) \
                            for std in std_per_channel], axis = 0)
            return np.clip(volume + noise, 0, 1)
# class RandomFlip():
#     def __init__(self, p):
#         self.p = p
#     def __call__(self, volume, mask):
#         if self.p > torch.rand(1):
            

######################################################
PATCH_SIZE = (2,32,32,32)
EXTRACTION_STEP = (1,8,8,8)
NUM_CLASS = 4

In [ ]:
TRAIN_PATH = "/content/data_train/*"
TEST_PATH = "/content/data_test/*"
PSEUDO_PATH = "/content/pseudo_data/*"
class IsegLoader(Dataset):
    def __init__(self, train_path=TRAIN_PATH, test_path= TEST_PATH, 
                 transform=None, typeData = "train"):
        list_testName = sorted(glob.glob(test_path))
        self.listName =  glob.glob(train_path) + glob.glob(PSEUDO_PATH) if typeData == "train" else list_testName
        self.transform = transform if typeData == "train" else None  # augment data bool
        self.typeData = typeData

    def __len__(self):
        return len(self.listName)
    def __getitem__(self, idx):
        subject = np.load(self.listName[idx])
        volume, mask = subject["image"], subject["mask"]
    ####################### augmentation data ##############################
        # if self.transform:
        #     volume = self.transform(volume)
        return torch.from_numpy(volume), torch.from_numpy(mask)



In [ ]:
# train_dataset = DataLoader(IsegLoader(), batch_size=16, 
#                         shuffle=True, num_workers=2, 
#                         drop_last=True, prefetch_factor = 8)
# test_dataset = DataLoader(IsegLoader(typeData="test"), batch_size=64,
#                           num_workers=2, prefetch_factor=32)

In [ ]:
# train_data = IsegLoader(transform = AddNoise(p = 0.2)) 

In [ ]:
# listOfT1_train = glob.glob("/content/drive/MyDrive/iseg2019/dataISEG/iSeg-2019-Training/*T1.hdr")

In [ ]:
# !mkdir train_pt
# !mkdir test_pt
# count_train = 0
# count_test = 0
# SAVE_TEST_PATH = "./test_pt/"
# SAVE_TRAIN_PATH = "./train_pt/"
# for i in range(len(listOfT1_train)):
#     print("\r read in: ", i+1,"th volume", end="")
#     t1 = np.transpose(nib.load(listOfT1_train[i]).get_fdata(), (-1,0,1,2))
#     t2 = np.transpose(nib.load(listOfT1_train[i].replace("T1", "T2")).get_fdata(), (-1,0,1,2))
#     seg = np.transpose(nib.load(listOfT1_train[i].replace("T1", "label")).get_fdata(), (-1,0,1,2))
#     t1 = min_max_preprocess(t1)
#     t2 = min_max_preprocess(t2)
    
#     t1 = np.asarray(t1, np.float32)
#     t2 = np.asarray(t2, np.float32)
#     seg = np.asarray(seg, np.int64)

#     inputs = np.concatenate([t1, t2], axis= 0)

#     label_patch = extract_patches(seg, (1,*PATCH_SIZE[1:]), EXTRACTION_STEP)
#     valid_index = np.where(np.sum(label_patch, axis=(1, 2, 3, 4)) != 0)
#     y = label_patch[valid_index]
#     del label_patch, seg

#     input_patches = extract_patches(inputs, PATCH_SIZE, EXTRACTION_STEP)
#     data_input = input_patches[valid_index]
#     del input_patches, inputs

#     x_train, x_test, y_train, y_test = train_test_split(data_input, y, test_size=0.1)
#     del data_input, y
#     for j in range(x_train.shape[0]):
#         print("\rsave ",count_train,"th volume for train", end="")      
#         np.savez_compressed(SAVE_TRAIN_PATH+"iseg_"+str(count_train), 
#                                 image=x_train[j], mask=y_train[j]) 
#         count_train += 1 

#     for j in range(x_test.shape[0]):
#         print("\rsave ",count_test,"th volume for test", end="")      
#         np.savez_compressed(SAVE_TEST_PATH+"iseg_"+str(count_test), 
#                                 image=x_test[j], mask=y_test[j]) 
#         count_test += 1 
#     del x_train, x_test, y_train, y_test

In [ ]:
# import shutil
# shutil.make_archive("/content/drive/MyDrive/iseg2019/dataISEG_np/train_pt",
#                     "zip","/content/train_pt") 
# shutil.make_archive("/content/drive/MyDrive/iseg2019/dataISEG_np/test_pt",
#                     "zip","/content/test_pt") 

'/content/drive/MyDrive/iseg2019/dataISEG_np/test_pt.zip'

In [ ]:
# path_train = glob.glob("/content/data_train/*")
# path_test =  glob.glob("/content/data_test/*")
# from tqdm import tqdm
# SAVE_TEST_PATH = "./test_pt/"
# SAVE_TRAIN_PATH = "./train_pt/"
# count_train = 0
# count_test = 0
# for i in tqdm(path_train):
#     subject = np.load(i)
#     volume, mask = subject["image"], subject["mask"]
#     volume =  np.asarray(volume.transpose((-1,0,1,2)), np.float32)
#     mask = np.asarray(mask.transpose((-1,0,1,2)), np.int64)
#     np.savez_compressed(SAVE_TRAIN_PATH+"iseg_"+str(count_train), 
#                                 image=volume, mask=mask) 
#     count_train += 1 
# for i in tqdm(path_test):
#     subject = np.load(i)
#     volume, mask = subject["image"], subject["mask"]
#     volume =  np.asarray(volume.transpose((-1,0,1,2)), np.float32)
#     mask = np.asarray(mask.transpose((-1,0,1,2)), np.int64)
#     np.savez_compressed(SAVE_TEST_PATH+"iseg_"+str(count_test), 
#                                 image=volume, mask=mask) 
#     count_test += 1 

100%|██████████| 3977/3977 [01:12<00:00, 54.63it/s]


In [ ]:
# import time
# count = 1
# t1 = time.time()
# for _ in train_dataset:
#     print("\r read",count, end="")
#     count+=1
#     time.sleep(0.2)
# print("\n", time.time()-t1)

In [ ]:
# x, y = next(iter(train_dataset))

In [ ]:
# patches = volume
# seg_patches = mask
# for i in range(32):
#     plt.figure(i+1, figsize=(12,12))
#     plt.subplot(131),plt.imshow(patches[0,:,:,i]),plt.title('T1')
#     plt.subplot(132),plt.imshow(patches[1,:,:,i]),plt.title('T2')
#     plt.subplot(133),plt.imshow(seg_patches[0,:,:,i]),plt.title('seg')

##loss

In [ ]:
class SemiActiveLoss(nn.Module):
    def __init__(self, device, alpha =1e-9, beta = 1e-1, lamda = 1e-3):
        super().__init__()
        self.device = device
        self.alpha = alpha
        self.beta = beta     
        self.lamda = lamda 
    def LevelsetLoss(self, image, y_pred, kernel_size=5, smooth=1e-5):
        kernel = torch.ones(1, y_pred.size(1), kernel_size, kernel_size, kernel_size, device=self.device) / kernel_size**3
        padding = kernel_size //2
        lossRegion = 0.0
        y_pred_fuzzy = y_pred
        for ich in range(image.size(1)):
            target_ = image[:,ich:ich+1] 
            pcentroid_local = F.conv3d(target_ * y_pred_fuzzy + smooth, kernel, padding = padding) \
                                / F.conv3d(y_pred_fuzzy + smooth, kernel, padding = padding)
            plevel_local = target_ - pcentroid_local
            loss_local = plevel_local * plevel_local * y_pred_fuzzy

            pcentroid_global = torch.sum(target_ * y_pred_fuzzy, dim=(2,3,4),keepdim=True) \
                                / torch.sum(y_pred_fuzzy+smooth, dim=(2,3,4),keepdim = True)   
            plevel_global = target_ - pcentroid_global
            loss_global = plevel_global * plevel_global * y_pred_fuzzy

            lossRegion += torch.sum(loss_local) + self.beta * torch.sum(loss_global)
        return lossRegion 
    def GradientLoss(self, y_pred, penalty = "l1"):
        dH = torch.abs(y_pred[...,1:] - y_pred[...,:-1])
        dW = torch.abs(y_pred[:,:,:,1:] - y_pred[:,:,:,:-1])
        dD = torch.abs(y_pred[:,:,1:] - y_pred[:,:,:-1])
        if penalty == "l2":
            dH = dH * dH
            dW = dW * dW
            dD = dD * dD
        loss =  torch.sum(dH) +  torch.sum(dW)+ torch.sum(dD)
        return loss
    def ActiveContourLoss(self, y_true, y_pred, smooth=1e-5):   
        dim = (1,2,3,4)
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), y_true.size(4), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)[:,1:]
        y_pred = y_pred[:,1:]

        active = - torch.log(1-y_pred+smooth) * (1-yTrueOnehot) - torch.log(y_pred+smooth) * yTrueOnehot
        loss = torch.sum(active, dim = dim) / torch.sum(yTrueOnehot + y_pred- yTrueOnehot * y_pred +smooth, dim = dim)
        return torch.mean(loss)

    def forward(self, image, y_true, y_pred):
        active = self.ActiveContourLoss(y_true, y_pred)
        levelset =  self.LevelsetLoss(image, y_pred)
        length = self.GradientLoss(y_pred)
        return active + self.alpha * (levelset + self.lamda * length) 

##metrics

In [ ]:
def dice_CSF(y_true, y_pred, smooth = 1.0):
    output_standard = torch.argmax(y_pred, dim=1, keepdim = True)
    output_CSF = torch.where(output_standard == 1, 1, 0)
    label_CSF = torch.where(y_true == 1, 1, 0)

    intersection = torch.sum(label_CSF * output_CSF, dim=[1,2,3,4])
    union = torch.sum(label_CSF, dim=[1,2,3,4]) + torch.sum(output_CSF, dim=[1,2,3,4])
    return torch.mean((2. * intersection + smooth) / (union + smooth), dim=0)

def dice_GM(y_true, y_pred, smooth = 1.0):
    output_standard = torch.argmax(y_pred, dim=1, keepdim = True)
    output_GM = torch.where(output_standard == 2, 1.0, 0.0)
    label_GM = torch.where(y_true == 2, 1.0, 0.0)

    intersection = torch.sum(label_GM * output_GM, dim=[1,2,3,4])
    union = torch.sum(label_GM, dim=[1,2,3,4]) + torch.sum(output_GM, dim=[1,2,3,4])
    return torch.mean((2. * intersection + smooth) / (union + smooth), dim=0)

def dice_WM(y_true, y_pred, smooth = 1.0):
    output_standard = torch.argmax(y_pred, dim=1,keepdim = True)
    output_WM = torch.where(output_standard == 3, 1.0, 0.0)
    label_WM = torch.where(y_true == 3, 1.0, 0.0)

    intersection = torch.sum(label_WM * output_WM, dim=[1,2,3,4])
    union = torch.sum(label_WM, dim=[1,2,3,4]) + torch.sum(output_WM, dim=[1,2,3,4])
    return torch.mean((2. * intersection + smooth) / (union + smooth), dim=0)

##optimizer

In [ ]:
import math
import torch
from torch.optim.optimizer import Optimizer


class Nadam(Optimizer):
    """Implements Nadam algorithm (a variant of Adam based on Nesterov momentum).
    It has been proposed in `Incorporating Nesterov Momentum into Adam`__.
    Arguments:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float, optional): learning rate (default: 2e-3)
        betas (Tuple[float, float], optional): coefficients used for computing
            running averages of gradient and its square
        eps (float, optional): term added to the denominator to improve
            numerical stability (default: 1e-8)
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
        schedule_decay (float, optional): momentum schedule decay (default: 4e-3)
    __ http://cs229.stanford.edu/proj2015/054_report.pdf
    __ http://www.cs.toronto.edu/~fritz/absps/momentum.pdf
        Originally taken from: https://github.com/pytorch/pytorch/pull/1408
        NOTE: Has potential issues but does work well on some problems.
    """

    def __init__(self, params, lr=2e-3, betas=(0.9, 0.999), eps=1e-8,
                 weight_decay=0, schedule_decay=4e-3):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        defaults = dict(
            lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, schedule_decay=schedule_decay)
        super(Nadam, self).__init__(params, defaults)

    @torch.no_grad()
    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad
                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    state['m_schedule'] = 1.
                    state['exp_avg'] = torch.zeros_like(p)
                    state['exp_avg_sq'] = torch.zeros_like(p)

                # Warming momentum schedule
                m_schedule = state['m_schedule']
                schedule_decay = group['schedule_decay']
                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']
                eps = group['eps']
                state['step'] += 1
                t = state['step']
                bias_correction2 = 1 - beta2 ** t

                if group['weight_decay'] != 0:
                    grad = grad.add(p, alpha=group['weight_decay'])

                momentum_cache_t = beta1 * (1. - 0.5 * (0.96 ** (t * schedule_decay)))
                momentum_cache_t_1 = beta1 * (1. - 0.5 * (0.96 ** ((t + 1) * schedule_decay)))
                m_schedule_new = m_schedule * momentum_cache_t
                m_schedule_next = m_schedule * momentum_cache_t * momentum_cache_t_1
                state['m_schedule'] = m_schedule_new

                # Decay the first and second moment running average coefficient
                exp_avg.mul_(beta1).add_(grad, alpha=1. - beta1)
                exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1. - beta2)

                denom = (exp_avg_sq.sqrt() / math.sqrt(bias_correction2)).add_(eps)
                p.addcdiv_(grad, denom, value=-group['lr'] * (1. - momentum_cache_t) / (1. - m_schedule_new))
                p.addcdiv_(exp_avg, denom, value=-group['lr'] * momentum_cache_t_1 / (1. - m_schedule_next))

        return loss